In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
# from art.estimators.classification import PyTorchClassifier

import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./RSLAD_100.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    


        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./RSLAD_100.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def10/y_pred_{attack_name}{eps}_Def10.npy", all_preds)


In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np

def attack_pgd(model,train_batch_data,train_batch_labels,attack_iters=10,step_size=2/255.0,epsilon=8.0/255.0):
    ce_loss = torch.nn.CrossEntropyLoss().cuda()
    train_ifgsm_data = train_batch_data.detach() + torch.zeros_like(train_batch_data).uniform_(-epsilon,epsilon)
    train_ifgsm_data = torch.clamp(train_ifgsm_data,0,1)
    for i in range(attack_iters):
        train_ifgsm_data.requires_grad_()
        logits = model(train_ifgsm_data)
        loss = ce_loss(logits,train_batch_labels.cuda())
        loss.backward()
        train_grad = train_ifgsm_data.grad.detach()
        train_ifgsm_data = train_ifgsm_data + step_size*torch.sign(train_grad)
        train_ifgsm_data = torch.clamp(train_ifgsm_data.detach(),0,1)
        train_ifgsm_pert = train_ifgsm_data - train_batch_data
        train_ifgsm_pert = torch.clamp(train_ifgsm_pert,-epsilon,epsilon)
        train_ifgsm_data = train_batch_data + train_ifgsm_pert
        train_ifgsm_data = train_ifgsm_data.detach()
    return train_ifgsm_data

def rslad_inner_loss(model,
                teacher_logits,
                x_natural,
                y,
                optimizer,
                step_size=0.003,
                epsilon=0.031,
                perturb_steps=10,
                beta=6.0):
    # define KL-loss
    criterion_kl = nn.KLDivLoss(size_average=False,reduce=False)
    model.eval()
    batch_size = len(x_natural)
    # generate adversarial example
    x_adv = x_natural.detach() + 0.001 * torch.randn(x_natural.shape).cuda().detach()

    for _ in range(perturb_steps):
        x_adv.requires_grad_()
        with torch.enable_grad():
            loss_kl = criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                       F.softmax(teacher_logits, dim=1))
            loss_kl = torch.sum(loss_kl)
        grad = torch.autograd.grad(loss_kl, [x_adv])[0]
        x_adv = x_adv.detach() + step_size * torch.sign(grad.detach())
        x_adv = torch.min(torch.max(x_adv, x_natural - epsilon), x_natural + epsilon)
        x_adv = torch.clamp(x_adv, 0.0, 1.0)

    model.train()

    x_adv = Variable(torch.clamp(x_adv, 0.0, 1.0), requires_grad=False)
    # zero gradient
    optimizer.zero_grad()
    logits = model(x_adv)
    return logits
  

In [12]:
import os
import argparse
import torch
# from rslad_loss import *
# from cifar10_models import *
import torchvision
from torchvision import datasets, transforms
import time
# # we fix the random seed to 0, in the same computer, this method can make the results same as before.
# torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)
# torch.backends.cudnn.deterministic = True

# prefix = 'mobilenet_v2-CIFAR10_RSLAD'
epochs = 10
batch_size = 128
epsilon = 8/255.0
multiply = 1



student = DNNModel(input_shape,output_shape )
student = torch.nn.DataParallel(student)
student = student.cuda()
student.train()
optimizer = optim.SGD(student.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-4)
def kl_loss(a,b):
    loss = -a*b + torch.log(b+1e-5)*b
    return loss
teacher = DNNModel(input_shape,output_shape )
teacher.load_state_dict(torch.load('/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/dnn_pytorch.pt'))
teacher = torch.nn.DataParallel(teacher)
teacher = teacher.cuda()
teacher.eval()

/tmp/ipykernel_181/291553794.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load('/home/jovyan/Sample_Based_Extension/UNSW/transfer_attac

DataParallel(
  (module): DNNModel(
    (fc1): Linear(in_features=56, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=30, bias=True)
    (fc3): Linear(in_features=30, out_features=20, bias=True)
    (fc4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [13]:


# for epoch in range(0,epochs):
#     train_loss = 0.0
#     for step,(train_batch_data,train_batch_labels) in enumerate(train_loader):
#         student.train()
#         train_batch_data = train_batch_data.float().cuda()
#         train_batch_labels = train_batch_labels.cuda()
#         optimizer.zero_grad()
#         with torch.no_grad():
#             teacher_logits = teacher(train_batch_data)

#         adv_logits = rslad_inner_loss(student,teacher_logits,train_batch_data,train_batch_labels,optimizer,step_size=2/255.0,epsilon=epsilon,perturb_steps=3)
#         student.train()
#         nat_logits = student(train_batch_data)
#         kl_Loss1 = kl_loss(torch.log(F.softmax(adv_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#         kl_Loss2 = kl_loss(torch.log(F.softmax(nat_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#         # multiply 10 to keep consistent with CIFAR-10 dataset
#         kl_Loss1 = multiply*torch.mean(kl_Loss1)
#         kl_Loss2 = multiply*torch.mean(kl_Loss2)
#         loss = 5.0/6.0*kl_Loss1 + 1.0/6.0*kl_Loss2
#         loss.backward()
#         optimizer.step()
#         if step % 1000 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, step * len(train_batch_data), len(train_loader.dataset),
#                 100. * step / len(train_loader), loss.item()))
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
            
#             outputs = student(inputs)
#             kl_Loss1 = kl_loss(torch.log(F.softmax(adv_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#             kl_Loss2 = kl_loss(torch.log(F.softmax(nat_logits,dim=1)),F.softmax(teacher_logits.detach(),dim=1))
#             # multiply 10 to keep consistent with CIFAR-10 dataset
#             kl_Loss1 = multiply*torch.mean(kl_Loss1)
#             kl_Loss2 = multiply*torch.mean(kl_Loss2)
#             loss = 5.0/6.0*kl_Loss1 + 1.0/6.0*kl_Loss2
            
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
#     # print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}")

    
#     # if (epoch%2 == 0):
#     #     test_accs = []
#     #     student.eval()
#     #     for step,(test_batch_data,test_batch_labels) in enumerate(test_loader):
#     #         test_ifgsm_data = attack_pgd(student,test_batch_data,test_batch_labels,attack_iters=20,step_size=0.003,epsilon=8.0/255.0)
#     #         logits = student(test_ifgsm_data)
#     #         predictions = np.argmax(logits.cpu().detach().numpy(),axis=1)
#     #         predictions = predictions - test_batch_labels.cpu().detach().numpy()
#     #         test_accs = test_accs + predictions.tolist()
#     #     test_accs = np.array(test_accs)
#     #     test_acc = np.sum(test_accs==0)/len(test_accs)
#     #     print('robust acc',np.sum(test_accs==0)/len(test_accs))
#     #     # torch.save(student.state_dict(),'./models/'+prefix+str(np.sum(test_accs==0)/len(test_accs))+'.pth')


#     # # Early stopping check using min_delta
#     # if best_loss - avg_val_loss > min_delta:
#     #     best_loss = avg_val_loss
#     #     patience_counter = 0
#     # else:
#     #     patience_counter += 1

#     # if patience_counter >= patience:
#     #     print("Early stopping triggered")
#     #     break

    

In [14]:
student.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RSLAD/RSLAD_100.pt"))

/tmp/ipykernel_181/2390732642.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/R

<All keys matched successfully>

In [15]:
# calculate_performance_metrics(x_test, y_test, student, 'DNN', 'baseline', '0')

In [16]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./RSLAD_100.csv", mode='a', index=False, header=False)

In [15]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(201, 56) (201,)
Accuracy: 0.8308457711442786

macro
Precision: 0.8508196721311476
Recall: 0.8059015723912599
F1 Score: 0.8160529715762274

weighted
Precision: 0.8419704754913956
Recall: 0.8308457711442786
F1 Score: 0.8251362695566098

start XGB 100 BIM
(26, 56) (26,)
Accuracy: 0.8461538461538461

macro
Precision: 0.7777777777777778
Recall: 0.9047619047619048
F1 Score: 0.8045112781954887

weighted
Precision: 0.9145299145299146
Recall: 0.8461538461538461
F1 Score: 0.8600347021399654

start XGB 100 FGSM
(16, 56) (16,)
Accuracy: 0.5625

macro
Precision: 0.75
Recall: 0.6111111111111112
F1 Score: 0.5151515151515151

weighted
Precision: 0.78125
Recall: 0.5625
F1 Score: 0.4962121212121212

start XGB 100 PGD
(26, 56) (26,)
Accuracy: 0.8461538461538461

macro
Precision: 0.7777777777777778
Recall: 0.9047619047619048
F1 Score: 0.8045112781954887

weighted
Precision: 0.9145299145299146
Recall: 0.8461538461538461
F1 Score: 0.8600347021399654

start XGB 100 DF
(318, 56) (318,)

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8509615384615384

macro
Precision: 0.8705719163465642
Recall: 0.8299135556188848
F1 Score: 0.8397574492407863

weighted
Precision: 0.8617988525558947
Recall: 0.8509615384615384
F1 Score: 0.8466836134863421

start XGB 50 BIM
(93, 56) (93,)
Accuracy: 0.7526881720430108

macro
Precision: 0.5892857142857143
Recall: 0.8693181818181819
F1 Score: 0.5763517528223411

weighted
Precision: 0.955837173579109
Recall: 0.7526881720430108
F1 Score: 0.8202837994109342

start XGB 50 FGSM
(20, 56) (20,)
Accuracy: 0.5

macro
Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1 Score: 0.5

weighted
Precision: 0.8333333333333334
Recall: 0.5
F1 Score: 0.5

start XGB 50 PGD
(93, 56) (93,)
Accuracy: 0.7526881720430108

macro
Precision: 0.5892857142857143
Recall: 0.8693181818181819
F1 Score: 0.5763517528223411

weighted
Precision: 0.955837173579109
Recall: 0.7526881720430108
F1 Score: 0.8202837994109342

start XGB 50 DF
(294, 56) (294,)
Accuracy: 0.9047619047619048

macro
Precision: 0.6499278

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.6666666666666666

macro
Precision: 0.75
Recall: 0.75
F1 Score: 0.6666666666666666

weighted
Precision: 0.8333333333333334
Recall: 0.6666666666666666
F1 Score: 0.6666666666666666

start XGB 1 DF
(139, 56) (139,)
Accuracy: 0.9568345323741008

macro
Precision: 0.8383720930232558
Recall: 0.8383720930232558
F1 Score: 0.8383720930232558

weighted
Precision: 0.9568345323741008
Recall: 0.9568345323741008
F1 Score: 0.9568345323741008

start XGB 1 AutoPGD
(69, 56) (69,)
Accuracy: 0.8985507246376812

macro
Precision: 0.9426229508196722
Recall: 0.7666666666666666
F1 Score: 0.817391304347826

weighted
Precision: 0.910192444761226
Recall: 0.8985507246376812
F1 Score: 0.8862003780718337

start XGB 1 ZOO
(151, 56) (151,)
Accuracy: 0.4370860927152318

macro
Precision: 0.698581560283688
Recall: 0.5526315789473684
F1 Score: 0.3795020546289582

weighted
Precision: 0.776431355972007
Recall: 0.4370860927152318
F1 Score: 0.33068080494711755

start XGB 1 CaFA
(375, 56) (375,)
Accuracy: 0.448

macr

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9208333333333333

macro
Precision: 0.9363553113553114
Recall: 0.9072939668067526
F1 Score: 0.9164452588181402

weighted
Precision: 0.9281631562881562
Recall: 0.9208333333333333
F1 Score: 0.9194770193922736

start RF 50 BIM
(8, 56) (8,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 50 FGSM
(9, 56) (9,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 50 PGD
(8, 56) (8,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 50 DF
(273, 56) (273,)
Accuracy: 0.9157509157509157

macro
Precision: 0.6552371541501976
Recall: 0.6804917279411764
F1 Score: 0.6665958689534328

weighted
Precision: 0.9224594246333376
Recall: 0.9157509157509157
F1 Score: 0.9189181409220701

start RF 50 AutoPGD
(349, 56) (349,)
Accuracy: 0.9828080229226361

macro
Precision: 0.49566473988439

In [14]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
(210, 56) (210,)
Accuracy: 0.8952380952380953

macro
Precision: 0.9082389691526748
Recall: 0.6789933230611197
F1 Score: 0.7314578005115089

weighted
Precision: 0.8980643721760473
Recall: 0.8952380952380953
F1 Score: 0.8752648885641212

start XGB 100 BIM
(71, 56) (71,)
Accuracy: 0.7323943661971831

macro
Precision: 0.6922764227642276
Recall: 0.781547619047619
F1 Score: 0.690950744558992

weighted
Precision: 0.8418069391961525
Recall: 0.7323943661971831
F1 Score: 0.7563041479115241

start XGB 100 FGSM
(17, 56) (17,)
Accuracy: 0.9411764705882353

macro
Precision: 0.8333333333333333
Recall: 0.9666666666666667
F1 Score: 0.8827586206896552

weighted
Precision: 0.9607843137254901
Recall: 0.9411764705882353
F1 Score: 0.9460446247464503

start XGB 100 PGD
(71, 56) (71,)
Accuracy: 0.7323943661971831

macro
Precision: 0.6922764227642276
Recall: 0.781547619047619
F1 Score: 0.690950744558992

weighted
Precision: 0.8418069391961525
Recall: 0.7323943661971831
F1 Score: 0.756304

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.930921052631579

macro
Precision: 0.7421052631578947
Recall: 0.7113152804642167
F1 Score: 0.7253839205058716

weighted
Precision: 0.9266620498614957
Recall: 0.930921052631579
F1 Score: 0.9285765226073314

start XGB 50 AutoPGD
(453, 56) (453,)
Accuracy: 0.9801324503311258

macro
Precision: 0.9898876404494382
Recall: 0.7352941176470589
F1 Score: 0.8148921679909193

weighted
Precision: 0.9805342659424064
Recall: 0.9801324503311258
F1 Score: 0.976657771496869

start XGB 50 ZOO
(520, 56) (520,)
Accuracy: 0.9538461538461539

macro
Precision: 0.9757575757575758
Recall: 0.7551020408163265
F1 Score: 0.8254154775893906

weighted
Precision: 0.9560839160839161
Recall: 0.9538461538461539
F1 Score: 0.9469350860655208

start XGB 50 CaFA
(12, 56) (12,)
Accuracy: 0.9166666666666666

macro
Precision: 0.5
Recall: 0.4583333333333333
F1 Score: 0.4782608695652174

weighted
Precision: 1.0
Recall: 0.9166666666666666
F1 Score: 0.9565217391304349

start XGB 50 SINIFGSM
(5, 56) (5,)
Accuracy: 1.0

ma

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8

macro
Precision: 0.7387387387387387
Recall: 0.5949820788530467
F1 Score: 0.6078431372549019

weighted
Precision: 0.7783783783783783
Recall: 0.8
F1 Score: 0.7588235294117648

start XGB 20 BIM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input20/x_test_adv_BIM_Def10.npy not found
start XGB 20 FGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input20/x_test_adv_FGSM_Def10.npy not found
start XGB 20 PGD
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input20/x_test_adv_PGD_Def10.npy not found
start XGB 20 DF
(291, 56) (291,)
Accuracy: 0.9484536082474226

macro
Precision: 0.4842105263157895
Recall: 0.48936170212765956
F1 Score: 0.48677248677248675

weighted
Precision: 0.9384698860553445
Recall: 0.9484536082474226
F1 Score: 0.9434353351879124

start XGB 20 AutoPGD
(266, 56) (266,)
Accuracy: 0.9323308270676691

macro
Precision: 0.607283464566929

...

Accuracy: 0.9473684210526315

macro
Precision: 0.9651162790697674
Recall: 0.9117647058823529
F1 Score: 0.93354061406918

weighted
Precision: 0.9510403916768666
Recall: 0.9473684210526315
F1 Score: 0.9457729048622332

start DT 100 BIM
(332, 56) (332,)
Accuracy: 0.8102409638554217

macro
Precision: 0.7981950844854071
Recall: 0.7555642407536923
F1 Score: 0.7699085838751196

weighted
Precision: 0.8065684396572466
Recall: 0.8102409638554217
F1 Score: 0.8029869386791084

start DT 100 FGSM
(5029, 56) (5029,)
Accuracy: 0.5858023463909325

macro
Precision: 0.4397348884203127
Recall: 0.4681827046743553
F1 Score: 0.4321467785271462

weighted
Precision: 0.5034464422812281
Recall: 0.5858023463909325
F1 Score: 0.5254796139000654

start DT 100 PGD
(332, 56) (332,)
Accuracy: 0.8102409638554217

macro
Precision: 0.7981950844854071
Recall: 0.7555642407536923
F1 Score: 0.7699085838751196

weighted
Precision: 0.8065684396572466
Recall: 0.8102409638554217
F1 Score: 0.8029869386791084

start DT 100 DF
(383,

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7368421052631579

macro
Precision: 0.5
Recall: 0.3684210526315789
F1 Score: 0.42424242424242425

weighted
Precision: 1.0
Recall: 0.7368421052631579
F1 Score: 0.8484848484848485

start DT 20 SINIFGSM
(86, 56) (86,)
Accuracy: 0.5232558139534884

macro
Precision: 0.42452830188679247
Recall: 0.28846153846153844
F1 Score: 0.3435114503816794

weighted
Precision: 0.7700745941202283
Recall: 0.5232558139534883
F1 Score: 0.6231137937156045

start DT 20 VNIFGSM
(75, 56) (75,)
Accuracy: 0.72

macro
Precision: 0.4576923076923077
Recall: 0.4658808933002482
F1 Score: 0.4608010955152345

weighted
Precision: 0.6913846153846154
Recall: 0.72
F1 Score: 0.7050462170489559

start DT 1 baseline
(123, 56) (123,)
Accuracy: 0.7073170731707317

macro
Precision: 0.7091511936339523
Recall: 0.7088188559322034
F1 Score: 0.7072977260708619

weighted
Precision: 0.7104612796791098
Recall: 0.7073170731707317
F1 Score: 0.7072009905715134

start DT 1 BIM
(107, 56) (107,)
Accuracy: 0.35514018691588783

macro
Pr

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9894736842105263

macro
Precision: 0.5
Recall: 0.49473684210526314
F1 Score: 0.4973544973544973

weighted
Precision: 1.0
Recall: 0.9894736842105263
F1 Score: 0.9947089947089947

start DT 1 SINIFGSM
(97, 56) (97,)
Accuracy: 0.41237113402061853

macro
Precision: 0.5573529411764706
Recall: 0.5273109243697479
F1 Score: 0.3805042016806723

weighted
Precision: 0.6149484536082475
Recall: 0.41237113402061853
F1 Score: 0.3384977908689249

start DT 1 VNIFGSM
(464, 56) (464,)
Accuracy: 0.07543103448275862

macro
Precision: 0.24929665520475147
Recall: 0.44794911734164067
F1 Score: 0.0718052738336714

weighted
Precision: 0.40075292925590966
Recall: 0.07543103448275862
F1 Score: 0.02279499195635448



In [14]:
epsilon = 0
Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start XGB 100 baseline
(269, 56) (269,)
Accuracy: 0.8921933085501859

macro
Precision: 0.9429133858267716
Recall: 0.6704545454545454
F1 Score: 0.7239658893882028

weighted
Precision: 0.9045019465503614
Recall: 0.8921933085501859
F1 Score: 0.8689619030468644

start XGB 100 BIM
(25, 56) (25,)
Accuracy: 0.8

macro
Precision: 0.8076923076923077
Recall: 0.8529411764705883
F1 Score: 0.7947454844006567

weighted
Precision: 0.8769230769230769
Recall: 0.8
F1 Score: 0.806568144499179

start XGB 100 FGSM
(12, 56) (12,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 PGD
(25, 56) (25,)
Accuracy: 0.8

macro
Precision: 0.8076923076923077
Recall: 0.8529411764705883
F1 Score: 0.7947454844006567

weighted
Precision: 0.8769230769230769
Recall: 0.8
F1 Score: 0.806568144499179

start XGB 100 DF
(308, 56) (308,)
Accuracy: 0.9512987012987013

macro
Precision: 0.8340748980348536
Recall: 0.868657937806874
F1 Score: 0.8502673796791

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7818181818181819

macro
Precision: 0.6470588235294118
Recall: 0.88
F1 Score: 0.6590909090909091

weighted
Precision: 0.9358288770053476
Recall: 0.7818181818181819
F1 Score: 0.8264462809917356

start XGB 20 DF
(234, 56) (234,)
Accuracy: 0.9487179487179487

macro
Precision: 0.6576576576576576
Recall: 0.7323008849557522
F1 Score: 0.6866071428571429

weighted
Precision: 0.9598059598059597
Recall: 0.9487179487179487
F1 Score: 0.953617216117216

start XGB 20 AutoPGD
(440, 56) (440,)
Accuracy: 0.8954545454545455

macro
Precision: 0.7208538587848933
Recall: 0.807275496455978
F1 Score: 0.7533031056506265

weighted
Precision: 0.917696671144947
Recall: 0.8954545454545455
F1 Score: 0.9039666077182531

start XGB 20 ZOO
(482, 56) (482,)
Accuracy: 0.8340248962655602

macro
Precision: 0.5343325791855204
Recall: 0.5262634129456559
F1 Score: 0.5284910736121301

weighted
Precision: 0.8131508045286421
Recall: 0.8340248962655602
F1 Score: 0.8230004799842507

start XGB 20 CaFA
(21, 56) (21,)
Acc

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.52

macro
Precision: 0.5
Recall: 0.26
F1 Score: 0.34210526315789475

weighted
Precision: 1.0
Recall: 0.52
F1 Score: 0.6842105263157895

start XGB 1 baseline
(6, 56) (6,)
Accuracy: 0.8333333333333334

macro
Precision: 0.9
Recall: 0.75
F1 Score: 0.7777777777777777

weighted
Precision: 0.8666666666666667
Recall: 0.8333333333333334
F1 Score: 0.8148148148148148

start XGB 1 BIM
(38, 56) (38,)
Accuracy: 0.5526315789473685

macro
Precision: 0.5861538461538461
Recall: 0.5795454545454546
F1 Score: 0.5498257839721254

weighted
Precision: 0.602914979757085
Recall: 0.5526315789473685
F1 Score: 0.5442141940216394

start XGB 1 FGSM
(40, 56) (40,)
Accuracy: 0.525

macro
Precision: 0.5683760683760684
Recall: 0.5625
F1 Score: 0.5223130106851037

weighted
Precision: 0.593162393162393
Recall: 0.525
F1 Score: 0.5151477058453804

start XGB 1 PGD
(38, 56) (38,)
Accuracy: 0.5526315789473685

macro
Precision: 0.5861538461538461
Recall: 0.5795454545454546
F1 Score: 0.5498257839721254

weighted
Prec

...

Accuracy: 0.7142857142857143

macro
Precision: 0.35714285714285715
Recall: 0.5
F1 Score: 0.4166666666666667

weighted
Precision: 0.5102040816326531
Recall: 0.7142857142857143
F1 Score: 0.5952380952380952

start RF 20 AutoPGD
(366, 56) (366,)
Accuracy: 0.9289617486338798

macro
Precision: 0.6578780580555204
Recall: 0.6889124829364477
F1 Score: 0.6716356107660455

weighted
Precision: 0.9357388408983388
Recall: 0.9289617486338798
F1 Score: 0.9321386145334827

start RF 20 ZOO
(410, 56) (410,)
Accuracy: 0.9048780487804878

macro
Precision: 0.7338789682539683
Recall: 0.6869918699186992
F1 Score: 0.7067722946580843

weighted
Precision: 0.8959821428571428
Recall: 0.9048780487804878
F1 Score: 0.8995873906585244

start RF 20 CaFA
(4, 56) (4,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 20 SINIFGSM
(13, 56) (13,)
Accuracy: 0.9230769230769231

macro
Precision: 0.5
Recall: 0.46153846153846156
F1 Score: 0.48

weighted
Pre

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.9266666666666666

macro
Precision: 0.9403547323811019
Recall: 0.7538684281965433
F1 Score: 0.8127340823970037

weighted
Precision: 0.9288222675665778
Recall: 0.9266666666666666
F1 Score: 0.9169288389513108

start DT 100 BIM
(249, 56) (249,)
Accuracy: 0.3614457831325301

macro
Precision: 0.40789127287087007
Recall: 0.3291208791208791
F1 Score: 0.3099848371298604

weighted
Precision: 0.6348106655924739
Recall: 0.3614457831325301
F1 Score: 0.43939398075422104

start DT 100 FGSM
(118, 56) (118,)
Accuracy: 0.3474576271186441

macro
Precision: 0.32938249926836405
Recall: 0.3239734299516908
F1 Score: 0.3261143662389676

weighted
Precision: 0.3587545820250691
Recall: 0.3474576271186441
F1 Score: 0.3525393558995194

start DT 100 PGD
(249, 56) (249,)
Accuracy: 0.3614457831325301

macro
Precision: 0.40789127287087007
Recall: 0.3291208791208791
F1 Score: 0.3099848371298604

weighted
Precision: 0.6348106655924739
Recall: 0.3614457831325301
F1 Score: 0.43939398075422104

start DT 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9223744292237442

macro
Precision: 0.5
Recall: 0.4611872146118721
F1 Score: 0.47980997624703087

weighted
Precision: 1.0
Recall: 0.9223744292237442
F1 Score: 0.9596199524940617

start DT 1 SINIFGSM
(235, 56) (235,)
Accuracy: 0.30638297872340425

macro
Precision: 0.325
Recall: 0.3116929698708752
F1 Score: 0.3022895757818619

weighted
Precision: 0.36276595744680856
Recall: 0.30638297872340425
F1 Score: 0.3184357762735012

start DT 1 VNIFGSM
(266, 56) (266,)
Accuracy: 0.32706766917293234

macro
Precision: 0.32615332428765265
Recall: 0.3154815481548155
F1 Score: 0.3165487246472505

weighted
Precision: 0.35541236061660253
Recall: 0.32706766917293234
F1 Score: 0.3369491019091789



In [17]:
epsilon = 0

Def = "Def10"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9651567944250871

macro
Precision: 0.9681528662420382
Recall: 0.9642857142857143
F1 Score: 0.9650341130604287

weighted
Precision: 0.9673761068820881
Recall: 0.9651567944250871
F1 Score: 0.9650846289164645

start BIM
Accuracy: 0.4702549575070821

macro
Precision: 0.5419140754982068
Recall: 0.5914602662363856
F1 Score: 0.42536114483032117

weighted
Precision: 0.8267500671111697
Recall: 0.4702549575070821
F1 Score: 0.5453307591118006

start FGSM
Accuracy: 0.4624145785876993

macro
Precision: 0.5344807219807219
Recall: 0.5357107903107414
F1 Score: 0.46218853820598005

weighted
Precision: 0.6150252351846884
Recall: 0.4624145785876993
F1 Score: 0.4664833054586458

start PGD
Accuracy: 0.4702549575070821

macro
Precision: 0.5419140754982068
Recall: 0.5914602662363856
F1 Score: 0.42536114483032117

weighted
Precision: 0.8267500671111697
Recall: 0.4702549575070821
F1 Score: 0.5453307591118006

start DF
Accuracy: 0.8825757575757576

macro
Precision: 0.7612927923001203


In [15]:

# torch.save(student.state_dict(), "./RSLAD_100.pt")

In [17]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")